<a href="https://colab.research.google.com/github/ichekhovskikh/recommendation-system/blob/master/RecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install pymorphy2[fast]

import gensim
import os
import collections
import smart_open
import random
import json
import urllib.request
import pymorphy2
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from pymystem3 import Mystem

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Начинаем
Для начала нам понадобится комплект документов для обучения нашей модели doc2vec. Теоретически, документ может быть чем угодно: коротким твитом из 140 символов, отдельным абзацем, новостной статьей или книгой. В NLP комплект документов часто называют корпусом.

Будем тренировать нашу модель на собственном корпусе. Этот корпус содержит 70 текстов.

И мы проверим нашу модель на глаз, используя тестовый корпус, который содержит 7 документов.

Dataset состоит из трех строк: id (идентификатор строки), text (текст статьи), tag (идентификатор самой статьи, вектор которого будем обучать)

In [0]:
train_path = 'https://raw.githubusercontent.com/ichekhovskikh/recommendation-system/master/train.json'
test_path = 'https://raw.githubusercontent.com/ichekhovskikh/recommendation-system/master/test.json'

## Опредлим функцию для чтения и предварительной обработки текста
Ниже мы определяем функцию для открытия  train/test файла, предварительно обрабатываем каждый текст датасета, используя простой инструмент предварительной обработки gensim (то есть, разбиваем текст на отдельные слова, удалите знаки препинания, установите строчные буквы и т. д.), лемматизацию, удаление стоп слов и возвращаем список слов. Для обучения модели нам нужно будет связать тег с каждым документом учебного корпуса. В нашем случае тег - это идентификатор статьи.

Лемматизация каждого слова статьи:

In [0]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(words):
    for word in words:
        yield morph.parse(word)[0].normal_form

Удаление стоп слов:

In [0]:
russian_stopwords = stopwords.words("russian")

def remove_stopwords(words):
    return [word for word in words if word not in russian_stopwords]

Предобработка текста статьи:

In [0]:
def advanced_preprocess(text):
    normalized_text = gensim.utils.simple_preprocess(text)
    normalized_text = remove_stopwords(normalized_text)
    normalized_text = list(lemmatize(normalized_text))
    return normalized_text

Отрытие файла с корпусом статей:

In [0]:
def read_corpus(corpus_path, tokens_only=False):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        for article in corpus:
            normalized_text = advanced_preprocess(article['text'])
            if tokens_only:
                yield normalized_text
            else:
                yield gensim.models.doc2vec.TaggedDocument(normalized_text, article['tags'])

Получение исходного текста статьи:

In [0]:
def get_article_text(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['text']

In [0]:
train_corpus = list(read_corpus(train_path))
test_corpus = list(read_corpus(test_path, tokens_only=True))

Давайте посмотрим на учебный корпус:

In [34]:
print(train_corpus[:2])

[TaggedDocument(words=['относиться', 'семья', 'язык', 'подобный', 'синтаксис', 'синтаксис', 'наиболее', 'близкий', 'java', 'язык', 'иметь', 'статический', 'типизация', 'поддерживать', 'полиморфизм', 'перегрузка', 'оператор', 'число', 'оператор', 'явный', 'неявный', 'приведение', 'тип', 'делегат', 'атрибут', 'событие', 'свойство', 'обобщённый', 'тип', 'метод', 'итератор', 'анонимный', 'функция', 'поддержка', 'замыкание', 'linq', 'исключение', 'комментарий', 'формат', 'xml', 'перенять', 'многий', 'свой', 'язык', 'pascal', 'модула', 'smalltalk', 'особенность', 'java', 'опираться', 'практика', 'использование', 'исключать', 'некоторый', 'модель', 'проблематичный', 'разработка', 'программный', 'система', 'например', 'отличие', 'некоторый', 'другой', 'язык', 'поддерживать', 'множественный', 'наследование', 'класс', 'допускаться', 'множественный', 'наследование', 'интерфейс'], tags=['0']), TaggedDocument(words=['программа', 'java', 'транслироваться', 'байт', 'код', 'java', 'выполнять', 'виртуа

Корпус тестирования выглядит так:

In [35]:
print(test_corpus[:2])

[['язык', 'ассемблер', 'машинный', 'ориентировать', 'язык', 'низкий', 'уровень', 'команда', 'прямо', 'соответствовать', 'отдельный', 'команда', 'машина', 'также', 'предоставлять', 'дополнительный', 'возможность', 'облегчение', 'такой', 'макрокоманда', 'выражение', 'средство', 'обеспечение', 'модульность', 'программа', 'рассматриваться', 'автокод', 'расширить', 'конструкция', 'язык', 'высокий', 'уровень', 'являться', 'существенно', 'платформо', 'зависимый', 'язык', 'ассемблер', 'различный', 'аппаратный', 'платформа', 'несовместимый', 'хотя', 'мочь', 'целое', 'подобный', 'русский', 'язык', 'именоваться', 'просто', 'ассемблер', 'типичный', 'выражение', 'тип', 'писать', 'программа', 'ассемблер', 'строго', 'говорить', 'неверный', 'ассемблер', 'именоваться', 'утилит', 'трансляция', 'программа', 'язык', 'ассемблер', 'объектный', 'код', 'компьютер'], ['перитонит', 'развиваться', 'вследствие', 'бактериальный', 'инфицирование', 'брюшной', 'полость', 'подавлять', 'большинство', 'пациент', 'перито

Обратите внимание, что корпус тестирования представляет собой просто список списков и не содержит никаких тегов.

# Обучение модели
## Создание объекта Doc2Vec
Теперь мы создадим модель Doc2Vec с векторным размером 80 слов и перебираем учебный корпус 100 раз. Мы устанавливаем минимальную длину слова равной двум, чтобы отбрасывать слова с очень малым количеством вхождений.

Однако это очень маленький набор данных (70 документов) с короткими текстами (несколько сотен слов). 

Словарь содержит в себе все уникальные слова, извлеченных из учебного корпуса.

In [36]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=80, min_count=2, epochs=100)
model.build_vocab(train_corpus)
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 1.24 s, sys: 71.2 ms, total: 1.31 s
Wall time: 1.3 s


## Inferring a Vector
Важно отметить, что теперь вы можете вывести вектор для любого фрагмента текста без необходимости переобучать модель, передав список слов в функцию model.infer_vector. Затем этот вектор можно сравнить с другими векторами по косинусной близости.

In [37]:
text = 'Горностай небольшой зверек семейства куньих очень ценится'
model.infer_vector(advanced_preprocess(text))

array([-0.04096178, -0.18379952, -0.11860437, -0.20194705, -0.50240123,
        0.1839197 , -0.15432355,  0.72807366,  0.5409689 , -0.4779142 ,
       -0.10427655,  0.20272826,  0.00589776,  0.05693987,  0.16688912,
       -0.42739728, -0.01697599, -0.07323613,  0.26289058,  0.64651096,
        0.17578113, -0.28664327, -0.07712755,  0.03304586, -0.3750441 ,
       -0.1947108 , -0.13230076,  0.13952   , -0.02209855, -0.19145404,
        0.31799352, -0.17331338, -0.15030901, -0.04337816,  0.37577283,
       -0.14438367, -0.15898366,  0.16706972,  0.08218319, -0.24977092,
       -0.43249604, -0.38224697,  0.16896726, -0.02178737,  0.284938  ,
       -0.21531668,  0.13852073, -0.21743523, -0.2834976 ,  0.04890248,
       -0.03676964,  0.3413355 ,  0.1220827 ,  0.3642785 ,  0.07248931,
       -0.32313284, -0.81035566,  0.074465  ,  0.23671976,  0.09117885,
       -0.78747183,  0.03215976, -0.19274847,  0.43228692, -0.01104711,
        0.29571605,  0.00713215, -0.5716812 ,  0.19893852,  0.33

Обратите внимание, что, поскольку лежащие в основе алгоритмы обучения представляют собой подходы на основе итеративной аппроксимации, в которой используется внутренняя рандомизация, таким обрахрм, повторные выводы одного и того же текста будут возвращать слегка разные векторы.

# Оценочная модель
Чтобы оценить нашу новую модель, мы сначала выведем новые векторы для каждого документа тренировочного корпуса, сравним выведенные векторы с тренировочным корпусом.

Проверка выведенного вектора по обучающему вектору является своего рода «проверкой работоспособности» в отношении того, ведет ли модель себя адекватно, хотя и не является реальным значением «точности».

Можем взглянуть на пример:

In [39]:
doc_id = random.randint(0, len(train_corpus) - 1)

inferred_vector = model.infer_vector(train_corpus[doc_id].words)
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}): «{}»\n'.format(doc_id, get_article_text(doc_id, train_path)))
for label, index in [('1)', 0), ('2', 1), ('3)', 2), ('4)', 3)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], get_article_text(int(sims[index][0]), train_path)))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


ТЕКСТ ИСХДНОГО ДОКУМЕНТА (30): «Ка-92 являет собой концептуальную разработку отечественных конструкторов. Данная машина является детищем конструкторского бюро Камова. Ка-92 проектируется как скоростной вертолет нового поколения. Данный аппарат имеет большой интерес у заказчиков в Российской Федерации и за ее рубежами. Главным конструктором вертолета модели Ка-92 является 70-летний Сергей Михеев. На проектирование вертолета модели Ка-92 Российская Федерация выделила почти 7,5 миллиардов рублей. Главной задачей конструкторов является изготовление конкурентоспособного аппарата, который был бы экономичным и надежным. Эти характеристики позволят продавать Ка-92 во все страны мира.»

1) ('30', 0.9038764238357544): «Ка-92 являет собой концептуальную разработку отечественных конструкторов. Данная машина является детищем конструкторского бюро Камова. Ка-92 проектируется как скоростной вертолет нового поколения. Данный аппарат имеет большой интерес у заказчиков в Российской Федерации и за ее руб

Обратите внимание, что наиболее похожий документ (как правило, тот же текст) имеет параметр сходства, приближающийся к единице. 

# Тестирование модели
Используя тот же подход, что и выше, мы выведем вектор для случайно выбранного тестового документа и сравним документ с нашей моделью на глаз.

In [40]:
doc_id = random.randint(0, len(test_corpus) - 1)

inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}): «{}»\n'.format(doc_id, get_article_text(doc_id, test_path)))
for label, index in [('1)', 0), ('2', 1), ('3)', 2), ('4)', 3)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], get_article_text(int(sims[index][0]), train_path)))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


ТЕКСТ ИСХДНОГО ДОКУМЕНТА (4): «Начав с подражания классикам американского рок-н-ролла 1950-х годов, The Beatles пришли к собственному стилю и звучанию. The Beatles оказали значительное влияние на рок-музыку и признаются специалистами одной из наиболее успешных групп XX века, как в творческом, так и в коммерческом смысле. Многие известные рок-музыканты признают, что стали таковыми под влиянием песен The Beatles. С момента выпуска сингла «Please Please Me / Ask Me Why» в 1963 году группа начала восхождение к успеху, породив своим творчеством глобальное явление — битломанию. Четвёрка стала первой британской группой, пластинки которой завоевали популярность и первые места в чартах США, и с неё началось всемирное признание британских коллективов, а также «ливерпульского» (Merseybeat) звучания рок-музыки. Музыкантам группы и их продюсеру и звукорежиссёру Джорджу Мартину принадлежат новаторские разработки в области звукозаписи, комбинирования различных стилей, включая симфоническую и психодел